In [1]:
# Process Return A and LEOKA data by state, year and agency type
# Requires these files from Open ICPSR:
    # https://www.openicpsr.org/openicpsr/project/100707/version/V20/view?path=/openicpsr/100707/fcr:versions/V20/ucr_offenses_known_yearly_1960_2022_dta.zip&type=file
    # https://www.openicpsr.org/openicpsr/project/102180/version/V13/view?path=/openicpsr/102180/fcr:versions/V13/ucr_leoka_yearly_1960_2022_rds.zip&type=fil

In [2]:
import pandas as pd
import numpy as np
import pyreadr


In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
# Define the columns we need from the LEOKA (Law Enforcement Officers Killed and Assaulted) data
# These columns contain agency information, staffing data, and incident counts that we'll use for analysis
keepcols = [
    "ori",                           # Original agency identifier (7 characters)
    "ori9",                          # Extended agency identifier (9 characters) 
    "fips_state_code",               # Federal Information Processing Standard state code
    "fips_county_code",              # FIPS county code
    "agency_name",                   # Name of the law enforcement agency
    "state",                         # Full state name
    "state_abb",                     # State abbreviation
    "year",                          # Year of the data
    "agency_type",                   # Type of agency (sheriff, local police, etc.)
    'population_group',              # Population size category served by agency
    "msa",                           # Metropolitan Statistical Area code
    "population",                    # Population served by the agency
    "covered_by",                    # Whether agency is covered by another agency
    "total_employees_officers",      # Number of sworn officers
    "total_employees_civilians",     # Number of civilian employees
    "total_employees_total",         # Total employees (officers + civilians)
    "officers_killed_total",         # Number of officers killed
    "assaults_no_injury_total"       # Number of assaults on officers with no injury
]

In [17]:
# Load and clean the LEOKA (Law Enforcement Officers Killed and Assaulted) data
# This data comes from the FBI's Uniform Crime Reporting program and contains information about
# law enforcement agencies, their staffing levels, and incidents involving officers
# Source: https://www.icpsr.umich.edu/web/NACJD/studies/38800/datadocumentation
leoka = (
    pyreadr
    .read_r(
        # Using Jacob Kaplan's concatenated files for easier processing
        f"../data/leoka_yearly_1960_2023.rds",
    )
    [None]  # Extract the single dataframe from the R data structure
    [keepcols]  # Select only the columns we need
    .assign(
        # Convert data types to ensure consistency and proper processing
        ori=lambda x: x.ori.astype(str),
        ori9=lambda x: x.ori9.astype(str),
        year = lambda x: x.year.astype(int).astype(str),
        agency_name=lambda x: x.agency_name.astype(str),
        state=lambda x: x.state.astype(str),
        state_abb=lambda x: x.state_abb.astype(str),
        # agency_type=lambda x: x.agency_type.astype(str),  # Commented out - already string
        msa=lambda x: x.msa.astype(str),
    )
)
leoka.head(3)

,ori,ori9,fips_state_code,fips_county_code,agency_name,state,state_abb,year,agency_type,population_group,msa,population,covered_by,total_employees_officers,total_employees_civilians,total_employees_total,officers_killed_total,assaults_no_injury_total
0,AK00101,AK0010100,02,020,anchorage,alaska,AK,2023,local police department,"city 250,000 thru 499,999",39,285026.0,"no, it is not covered by another agency",400.0,166.0,566.0,0.0,519.0
1,AK00101,AK0010100,02,020,anchorage,alaska,AK,2022,local police department,"city 250,000 thru 499,999",38,285821.0,"no, it is not covered by another agency",406.0,157.0,563.0,0.0,536.0
2,AK00101,AK0010100,02,020,anchorage,alaska,AK,2021,local police department,"city 250,000 thru 499,999",38,286238.0,"no, it is not covered by another agency",413.0,161.0,574.0,0.0,0.0


In [18]:
# Check the total number of unique agencies (using ori9 which is the extended identifier)
# This gives us a sense of the scope of law enforcement agencies covered in the dataset
len(leoka["ori9"].unique())

22033

In [19]:
# Get basic counts of our two main agency types for comparison
# This helps us understand the relative scale of sheriff's offices vs local police departments
sheriffs = leoka[leoka["agency_type"] == "sheriffs office"]
police = leoka[leoka["agency_type"] == "local police department"]

print(f"Sheriff's offices: {sheriffs['ori'].nunique()}")
print(f"Local police departments: {police['ori'].nunique()}")
print(f"Ratio (police to sheriff): {police['ori'].nunique() / sheriffs['ori'].nunique():.1f}:1")

Sheriff's offices: 3063
Local police departments: 14263
Ratio (police to sheriff): 4.7:1


In [20]:
# Calculate what percentage of sheriff's offices serve populations greater than 10,000
# This helps us understand the size distribution of sheriff's offices
# Many sheriff's offices serve small rural populations, so this filters to more substantial agencies

sheriff_agencies_over_10k = (
    leoka
    .loc[lambda x: x.agency_type == "sheriffs office"] 
    .loc[lambda x: x.population > 10000]
    ["ori9"].nunique()
)

total_sheriff_agencies = (
    leoka
    .loc[lambda x: x.agency_type == "sheriffs office"]
    ["ori9"].nunique()
)

percentage_over_10k = sheriff_agencies_over_10k / total_sheriff_agencies
print(f"Percentage of sheriff's offices serving >10,000 people: {percentage_over_10k:.1%}")

# Return the raw calculation for reference
percentage_over_10k

Percentage of sheriff's offices serving >10,000 people: 74.4%


0.743547860176413

In [21]:
# Data quality check: Verify that agency types don't change over time
# This is important because we need to trust that an agency classified as "sheriff's office" 
# in one year is the same type in all other years (agencies don't change types)
# If this assertion fails, it would indicate data quality issues

max_agency_types_per_ori = (
    leoka
    .groupby("ori")
    .agency_type
    .nunique()
    .max()
)

print(f"Maximum number of different agency types per ORI: {max_agency_types_per_ori}")
print("✓ Data quality check passed: Agency types are consistent over time" if max_agency_types_per_ori == 1 else "✗ Data quality check failed!")

assert max_agency_types_per_ori == 1, "Agency types should not change over time for the same ORI"

Maximum number of different agency types per ORI: 1
✓ Data quality check passed: Agency types are consistent over time


In [22]:
# Create a lookup table that maps ORI codes to agency types
# This is crucial for the analysis because we need to match agencies in the 
# Mapping Police Violence data (which has ORIs) to agency types
# 
# We filter out special jurisdictions, state agencies, federal agencies, and constables
# because our analysis focuses on the comparison between sheriff's offices and local police

type_lookup = (
    leoka
    .loc[
        # Filter to only sheriff's offices and local police departments
        lambda x: ~x.agency_type.isin([
            "special jurisdiction", 
            "state law enforcement agency", 
            "federal", 
            "constable/marshal", 
            np.nan
        ])
    ]
    .groupby(
        ["ori", "ori9", "year"]
    ).agg(
        agency_type=("agency_type", "first"),
        agency_name = ("agency_name", "first"),
        population_group = ("population_group", "first")
    )
    .reset_index()
)

# Save this lookup table for use in other notebooks
type_lookup.to_csv("../outputs/leoka_ori_type_lookup.csv", index=False)
print(f"Created lookup table with {len(type_lookup)} agency-year records")
type_lookup.head()

Created lookup table with 891774 agency-year records


,ori,ori9,year,agency_type,agency_name,population_group
0,AK00101,AK0010100,1960,local police department,anchorage,"city 25,000 thru 49,999"
1,AK00101,AK0010100,1961,local police department,anchorage,"city 25,000 thru 49,999"
2,AK00101,AK0010100,1962,local police department,anchorage,"city 25,000 thru 49,999"
3,AK00101,AK0010100,1963,local police department,anchorage,"city 50,000 thru 99,999"
4,AK00101,AK0010100,1964,local police department,anchorage,"city 50,000 thru 99,999"


In [23]:
# Calculate staffing levels by state, year, and agency type
# This creates a comprehensive table showing how many agencies of each type exist
# in each state for each year, plus their total staffing (officers and total employees)
# 
# This data is essential for understanding:
# - The relative size of sheriff's offices vs police departments by state
# - How staffing has changed over time
# - Which states rely more heavily on sheriffs vs local police

num_agencies = (
    leoka
    .groupby( ["year", "state_abb", "agency_type"] )
    .agg(
        agencies = ("ori", "nunique"),                    # Count of unique agencies
        officers = ("total_employees_officers", "sum"),  # Total sworn officers
        total_staff = ("total_employees_total", "sum")   # Total employees (officers + civilians)
    )
    .reset_index()
)

# Save this data for use in other notebooks
num_agencies.to_csv("../outputs/agency_staff_all_years.csv", index = False)
print(f"Created staffing data with {len(num_agencies)} agency-type-year records")

num_agencies.head(10)

Created staffing data with 12255 agency-type-year records


,year,state_abb,agency_type,agencies,officers,total_staff
0,1960,AK,local police department,6,110.0,138.0
1,1960,AK,state law enforcement agency,1,0.0,0.0
2,1960,AL,local police department,106,1815.0,2023.0
3,1960,AL,sheriffs office,67,0.0,0.0
4,1960,AR,local police department,66,731.0,780.0
5,1960,AR,sheriffs office,75,0.0,0.0
6,1960,AZ,local police department,30,1066.0,1224.0
7,1960,AZ,sheriffs office,14,0.0,0.0
8,1960,CA,local police department,354,15949.0,18934.0
9,1960,CA,sheriffs office,58,0.0,0.0


In [24]:
# Calculate the population served by each agency type by state and year
# This helps us understand what proportion of each state's population is served by
# sheriff's offices versus local police departments
# 
# This is important because it shows us the geographic/demographic reach of different
# agency types - for example, if sheriffs serve a large percentage of a state's 
# population, that suggests they have significant responsibility for public safety

agency_population = (
    leoka
    .groupby( ["year", "state_abb", "agency_type"])
    .agg(
        agency_population=("population", "sum"),  # Sum the population served by each agency type
    )
    .reset_index()
    .pivot(
        index=["year", "state_abb"],
        columns="agency_type", 
        values="agency_population"
    )
    .fillna(0)  # Fill missing values with 0 for agency types not present in a state
    .assign(
        # Calculate totals and percentages
        total_population=lambda x: x.sum(axis=1),
        other_pop = lambda x: x["federal"] + x["constable/marshal"] + x["special jurisdiction"] + x["state law enforcement agency"],
        percent_sheriff_pop=lambda x: (x["sheriffs office"]/x["total_population"] * 100).round(1),
        percent_police_pop=lambda x: (x["local police department"]/x["total_population"] * 100).round(1),
    )
    .sort_values("percent_sheriff_pop", ascending=False)  # Sort by sheriff percentage
    .drop(columns=["federal", "constable/marshal", "special jurisdiction", "state law enforcement agency"])
    .reset_index()
    .pipe(lambda x: x.rename_axis(None, axis=1))  # Clean up column names
)

# Save this population analysis for use in other notebooks
agency_population.to_csv("../outputs/agency_population.csv", index = False)
print(f"Created population data showing states where sheriffs serve the highest percentage of population")

agency_population.head()

Created population data showing states where sheriffs serve the highest percentage of population


,year,state_abb,local police department,sheriffs office,total_population,other_pop,percent_sheriff_pop,percent_police_pop
0,1971,SC,842786.0,1823483.0,2683742.0,17473.0,67.9,31.4
1,2021,WV,576898.0,1207480.0,1784378.0,0.0,67.7,32.3
2,2023,WV,577145.0,1192766.0,1769911.0,0.0,67.4,32.6
3,2022,WV,579590.0,1195365.0,1774955.0,0.0,67.3,32.7
4,2020,WV,590854.0,1205328.0,1796182.0,0.0,67.1,32.9


## Arrests


In [25]:
# Define the columns we need from the UCR Return A (arrests) data
# This data contains information about arrests made by different law enforcement agencies
# We need this to calculate arrest rates by agency type, which serves as our denominator
# for measuring police violence incidents per interaction with the public

usecols = [
    "ori",                      # Agency identifier
    "ori9",                     # Extended agency identifier
    "state_abb",               # State abbreviation
    "year",                    # Year of data
    "number_of_months_missing", # Data quality indicator
    "agency_type",             # Type of agency (sheriff, police, etc.)
    "actual_all_crimes",       # Total arrests for all crimes - our key metric
]

In [26]:
# Load UCR Return A data (arrests data)
# This dataset contains information about arrests made by law enforcement agencies
# We use arrests as a proxy for police-public interactions, which is our denominator
# for calculating incident rates (incidents per arrest)
# 
# Source: https://www.icpsr.umich.edu/web/NACJD/studies/38799/variables

arrests = (
    pyreadr
    .read_r(
        # Using Jacob Kaplan's concatenated files for easier processing
        f"../data/offenses_known_yearly_1960_2023.rds",
    )
    [None]  # Extract the single dataframe from the R data structure
    [usecols]  # Select only the columns we need
)

print(f"Loaded arrests data with {len(arrests)} agency-year records")
arrests.head(1)

Loaded arrests data with 1111562 agency-year records


,ori,ori9,state_abb,year,number_of_months_missing,agency_type,actual_all_crimes
0,AK00101,AK0010100,AK,2023.0,NaN,local police department,15773.0


In [27]:
# Create a summary table of total arrests by agency type and year
# This shows the overall trend in arrests over time for different agency types
# and helps us understand the relative volume of arrests by sheriffs vs police

type_and_year = (
    arrests
    .pivot_table(
        index = "year",
        columns = "agency_type",
        values = "actual_all_crimes",
        aggfunc = "sum"  # Sum all arrests for each agency type by year
    )
    .reset_index()
    .rename_axis(None, axis=1)  # Clean up column names
)

print("Annual arrests by agency type - shows national trends over time")
type_and_year.head()

Annual arrests by agency type - shows national trends over time


,year,constable/marshal,federal,local police department,sheriffs office,special jurisdiction,state law enforcement agency
0,1960.0,419.0,NaN,2421538.0,399575.0,2028.0,45711.0
1,1961.0,496.0,NaN,2694533.0,439181.0,2760.0,74682.0
2,1962.0,483.0,NaN,2683280.0,394278.0,2916.0,38579.0
3,1963.0,595.0,4.0,3194299.0,485087.0,3407.0,31055.0
4,1964.0,650.0,0.0,3591606.0,498924.0,3880.0,77492.0


In [28]:
# Create arrests data by state, year, and agency type
# This breaks down arrests by state to enable state-level analysis
# We group smaller agency types (federal, constable, special jurisdiction, state) 
# into an "other" category to focus on the sheriff vs police comparison
# 
# Source: https://www.icpsr.umich.edu/web/NACJD/studies/38799/variables

arrests_by_state = (
    arrests
    .pivot_table(
        index=["year", "state_abb"],
        columns="agency_type",
        values="actual_all_crimes",
        aggfunc="sum"
    )
    .reset_index()
    .assign(
        # Combine smaller agency types into "other" category
        other = lambda x: x["federal"] + x["constable/marshal"] + x["special jurisdiction"] + x["state law enforcement agency"],
        year = lambda x: x.year.astype(int).astype(str),  # Convert year to string for consistency
    )
    .drop(columns=["federal", "constable/marshal", "special jurisdiction", "state law enforcement agency"])
    .fillna(0)  # Fill missing values with 0
    .rename_axis(None, axis=1)  # Clean up column names
)

# Save this data for use in other notebooks
arrests_by_state.to_csv("../outputs/arrests_by_state.csv", index = False)
print(f"Created state-level arrests data with {len(arrests_by_state)} state-year records")
arrests_by_state.head()

Created state-level arrests data with 3255 state-year records


,year,state_abb,local police department,sheriffs office,other
0,1960,AK,2400.0,0.0,0.0
1,1960,AL,27850.0,4549.0,0.0
2,1960,AR,9241.0,3049.0,0.0
3,1960,AZ,32953.0,5449.0,0.0
4,1960,CA,410865.0,123944.0,0.0


In [29]:
# Create national-level arrests data by year and agency type
# This aggregates all arrests across the country by agency type for each year
# This is our key dataset for calculating national trends and rates
# 
# This data will be used to calculate:
# - Total arrests by sheriffs vs police over time
# - Arrest rates that serve as denominators for incident rate calculations

arrests_by_year = (
    arrests_by_state
    .drop(columns="state_abb")  # Remove state column to aggregate nationally
    .groupby("year")
    .sum()  # Sum arrests across all states for each year
)

# Save this data for use in other notebooks - this is a key output
arrests_by_year.to_csv("../outputs/arrests_by_year.csv")
print("Created national arrests data by year and agency type")
arrests_by_year.head()

Created national arrests data by year and agency type


,local police department,sheriffs office,other
year,,,
1960,2421538.0,399575.0,0.0
1961,2694533.0,439181.0,0.0
1962,2683280.0,394278.0,0.0
1963,3194299.0,485087.0,0.0
1964,3591606.0,498924.0,0.0


---

---

---